<a href="https://colab.research.google.com/github/neennera/GIBBIN-Web-Scraping/blob/main/WebScaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# Open google chrom by selenium
support ubuntu 20.04 due to colab update ubanto version

In [81]:
destination = "https://esc.kmutt.ac.th/index.php"

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(destination)
print(driver.title)



---

# Web Scraping : Go to 'Student' page
- Get HTML source code using BeautifulSoup
- Read and transfer code to lecture data using HTMLParser

In [84]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


X_path_login_username = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[1]/div[2]/div/input'
X_path_login_password = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[2]/div[2]/div/input'
X_path_login_submitButton = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[6]/div/input'

#username_input = driver.find_element("id","username")
username_input = driver.find_element("xpath",X_path_login_username)
username_input.send_keys(gibbon_username)

password_input = driver.find_element("xpath",X_path_login_password)
password_input.send_keys(gibbon_password)

login_box = driver.find_element("xpath",X_path_login_submitButton)
login_box.submit()

In [85]:
Xpath_of_learn_button = '//*[@id="header-menu"]/ul/li[3]'
Xpath_of_timetable_button = '//*[@id="header-menu"]/ul/li[3]/ul/li[4]/a'

driver.find_element('xpath',Xpath_of_learn_button)
timetable_page = driver.find_element('xpath',Xpath_of_timetable_button).get_property("href")

In [86]:
driver.get(timetable_page)

# Web Scraping : Read the class timetable

In [88]:
# from bs4 import BeautifulSoup
# import requests
import pandas as pd

In [89]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
  def __init__(self):
    HTMLParser.__init__(self)
    HTMLParser.unwanted_tag = ['div','i','a','br']
    HTMLParser.lectuteInWeek = []
    HTMLParser.day = 0

  def GetLectuteInWeek(self, HTMLtext):
    for i in range(0,7):
      HTMLParser.lectuteInWeek.append([])
      parser.feed(HTMLtext[i])
      HTMLParser.day += 1
    return HTMLParser.lectuteInWeek
  
  def handle_data(self, data):
    # print("Encountered some data  :", data)
    HTMLParser.lectuteInWeek[HTMLParser.day].append(data)

  # def handle_starttag(self, tag, attrs):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered a start tag:", tag)

  # def handle_endtag(self, tag):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered an end tag :", tag)

In [90]:
parser = MyHTMLParser()

In [91]:
Xpath_of_search_box = '//*[@id="search"]'
Xpath_of_search_submit_button = '//*[@id="ttView"]/div/div[2]/div/input'
Xpath_of_View_detail_button = '//*[@id="timetables"]/table/tbody/tr[1]/td[5]/nav/div/a'

def searching(classroom) :
  driver.get(timetable_page)
  driver.find_element('xpath',Xpath_of_search_box).clear()
  driver.find_element('xpath',Xpath_of_search_box).send_keys(classroom)
  driver.find_element('xpath',Xpath_of_search_submit_button).submit()
  driver.find_element('xpath',Xpath_of_View_detail_button).click()

def scaping():
  lecture = []
  for i in range(0,7):
    get_div = driver.find_element('xpath','//*[@id="ttWrapper"]/table/tbody/tr[2]/td[' + str(i+2) + ']')
    lecture.append(get_div.get_attribute('innerHTML'))
  parser = MyHTMLParser()
  lectureInWeek = parser.GetLectuteInWeek(lecture)
  return lectureInWeek

# Analyse Data

In [92]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta

In [93]:
class lecture:
  def __init__(self, day=-1 , start=-1, end=-1, subject="none"):
    self.day = day
    self.start = start
    self.end = end
    self.subject = subject

# datatime weekday() : mon = 0 ... sun = 6
# we will map to sun = 0 ... sat = 6 that we scaping from previous function
# set timenow to sunday of the week

timenow = datetime.utcnow() + timedelta(hours=7)
timenow = timenow + timedelta(days = 0) 
if(timenow.weekday() != 6) : 
  timenow = timenow + timedelta(days= - (timenow.weekday()+ 1)   )
timenow = datetime(timenow.year, timenow.month, timenow.day)

In [105]:
def analyseLecture(exampleOfLecture) :
  lectureData = []
  for dayInWeek in range(0,7):
    temp = exampleOfLecture[dayInWeek].copy()
    i=0
    while i< len(temp): 
      # Very spacial condition for M6 : 7XX class
      if temp[i][0:4] == "ESC7":  
        #get subject and class name
        subject = temp[i]
        className = "วิชาเลือก"

        # get time
        time = temp[i+1]
        start = time[0:5]
        end = time[-5:]

        startTime = timenow + timedelta(days = dayInWeek) + timedelta(hours=int(start[0:2])) + timedelta(minutes=int(start[3:5]))
        endTime = timenow + timedelta(days = dayInWeek) + timedelta(hours=int(end[0:2])) + timedelta(minutes=int(end[3:5]))
        #append data
        #print(str(dayInWeek) + "  " + subject + " ; " + className + " ; ", startTime.strftime('%Y-%m-%dT%H:%M:%S') + " " + endTime.strftime('%Y-%m-%dT%H:%M:%S'))
        lectureData.append(lecture(dayInWeek,startTime,endTime,subject+" "+className))

        # remove read class
        temp.remove(subject)
        temp.remove(time)
        continue

      # Common class
      if temp[i][0:4] == "ESC " and temp[i][0:5] != "ESC 7":  
        #get subject and class name
        subject = temp[i][0:3] + temp[i][4:-2]
        if subject[-1] == ".":
          subject = subject[:-1]
        className = temp[i+1]

        # get time
        timeIndex = temp.index(subject) +1
        time = temp[timeIndex]
        start = time[0:5]
        end = time[-5:]

        startTime = timenow + timedelta(days = dayInWeek) + timedelta(hours=int(start[0:2])) + timedelta(minutes=int(start[3:5]))
        endTime = timenow + timedelta(days = dayInWeek) + timedelta(hours=int(end[0:2])) + timedelta(minutes=int(end[3:5]))
      
        #append data
        #print(str(dayInWeek) + "  " + subject + " ; " + className + " ; ", startTime.strftime('%Y-%m-%dT%H:%M:%S') + " " + endTime.strftime('%Y-%m-%dT%H:%M:%S'))
        lectureData.append(lecture(dayInWeek,startTime,endTime,subject+" "+className))

        # remove read class
        temp.remove(subject)
        temp.remove(time)
        continue

      if temp[i] == "School Closed":
        print(str(dayInWeek) + "  " + "School Closed")
        lectureData.append(lecture(dayInWeek, "08:00" , "16:00", "School Closed"))
        break
      
      i+=1
  return lectureData

# google calendar function
using OAuth 2.0 Client IDs : "Desktop type"

OAuth consent screen : Internal, but add testing user

date 2023/02/05

In [ ]:
!sudo pip install --upgrade google-api-python-client

In [96]:
import pickle
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [97]:
# # OAuth API register
# scopes = ['https://www.googleapis.com/auth/calendar']
# flow = InstalledAppFlow.from_client_secrets_file("desktop.json", scopes=scopes)
# credentials = flow.run_console()
# pickle.dump(credentials, open("token.pkl", "wb"))

In [98]:
credentials = pickle.load(open("token.pkl", "rb"))
service = build("calendar", "v3", credentials=credentials)

In [175]:
## Update google calendar
API_KEY = '30068550143-uf6kotbrq4drn4903sh5pt1fnnqrjdk9.apps.googleusercontent.com' # API Key or Client ID (obtained from Google API Console)
CALENDAR_ID = 'c_6faf67346f38b30af4ae5e19ae9193da8135068afc9003d191ef17aae1fdf30c@group.calendar.google.com' # Calendar ID (can be found in the calendar's settings)

def add_event_to_calendar(CALENDAR_ID, event_summary, event_start_time, event_end_time):
    #service = build("calendar", "v3", credentials=credentials)
    # service = build('calendar', 'v3', developerKey=API_KEY)

    # Event details
    event = {
      'summary': event_summary,
      'description': 'update : ' + event_start_time.strftime('%Y-%m-%dT%H:%M:%S'),
      'start': {
        'dateTime': event_start_time.strftime('%Y-%m-%dT%H:%M:%S'),
        'timeZone': 'Asia/Bangkok',
      },
      'end': {
        'dateTime': event_end_time.strftime('%Y-%m-%dT%H:%M:%S'),
        'timeZone': 'Asia/Bangkok',
      },
    }

    #service.create_event(event)

    #print(event)

    # Call the Calendar API to add the event
    event = service.events().insert(calendarId=CALENDAR_ID, body=event).execute()

In [174]:
def clearEvent(CALENDAR_ID):
  page_token = None
  while True:
    events = service.events().list(calendarId=CALENDAR_ID, pageToken=page_token).execute()
    for event in events['items']:
      service.events().delete(calendarId=CALENDAR_ID, eventId=event["id"]).execute()
    page_token = events.get('nextPageToken')
    if not page_token:
      break

# Working Space

In [165]:
#ESCK_Class = ['4E','4S','4C','5E','5S','5C','6E','6S','6C','AU1/1','AU1/2','BI1/1','BI1/2','AU2/1','AU2/2','AU3/1']
ESC_Class = ['4E','4S','4C','5E','5S','5C','6E','6S','6C']
calendar_id ={
    '4E' : "c_7493eafab5255a71d0f49cdf93fe528a6601e79fe11cf1c0cf7a437af67f6125@group.calendar.google.com",
    '4S' : "c_92d7fdce936e171e0dfe52f707f3b94bfcd960bc3d1ca3a3225fc8baa3f4744c@group.calendar.google.com",
    '4C' : "c_1e366f9adb792570cbc8bef95598e778198759504ed7de4908c4e9fcb41ae643@group.calendar.google.com",
    '5E' : "c_bd5ea29ccaa8769077af807117c6c5f09c782c8095a96e0eefe4a095610925a1@group.calendar.google.com",
    '5S' : "c_73399ff64a3cf9502c8f5b89b524ddcfedb5964eca2f3880c479c89cf536d1d1@group.calendar.google.com",
    '5C' : "c_e6cc713d9a9040a17a623f86d905dd608e9aa23a47f2b6f0590a2ceeb069604f@group.calendar.google.com",
    '6E' : "c_02610126a1fd11d5d0f8240243ff52c8aafd128e7820d912bc268eb4b3cd28f8@group.calendar.google.com",
    '6S' : "c_8235ec98b028aad723711a32525d0578acadf19ea40de1653d3b0a6d44f8d175@group.calendar.google.com",
    '6C' : "c_448a21fe4dd324045fcfaa4f25396637576dc645ebab2979d3fef73db3de55ba@group.calendar.google.com"
}

In [178]:
for classroom in ESC_Class:
  searching(classroom)   #goToPages
  CALENDAR_ID = calendar_id[classroom]
  exampleOfLecture = scaping()  #getData
  analysedLecture = analyseLecture(exampleOfLecture) #analyseData to day / str /end / subject
  service = build("calendar", "v3", credentials=credentials)
  clearEvent(CALENDAR_ID)
  for item in analysedLecture:
    add_event_to_calendar(CALENDAR_ID,item.subject, item.start, item.end)